In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = 'retina'
from matplotlib import rcParams
from EC_CV import *
import cv2

rcParams['figure.figsize'] = (20,28)

img = plt.imread('TheWall.bmp')
plt.imshow(img)
np.shape(img)

In [ ]:
#splitting the image in two separate parts, right at the piece of litter.

imgL = img[:,:870]
imgR = img[:,870:]
print(np.shape(imgL))
print(np.shape(imgR))

fig, ax = plt.subplots(1,2)
ax[0].imshow(imgL)
ax[1].imshow(imgR)

In [ ]:
#looking at the two images put together in a vertical 
#straight line at the middle of their overlap

overlap = 70 # also try 52 and 46
heightL, widthL, temp = imgL.shape
heightR, widthR, temp = imgR.shape
    
stitch = np.concatenate((imgL[:,:widthL-int(overlap/2)], imgR[:,int(overlap/2):]), axis=1)
plt.imshow(stitch)

In [ ]:
#zooming in to appreciate the splitting artifact

rcParams['figure.figsize'] = (20,10)
tiny_stitch = stitch[int(heightL/2)-overlap*2:int(heightL/2)+overlap*2,widthL-overlap*2:widthL+overlap]
plt.imshow(tiny_stitch)

In [ ]:
#using a section of 100 vertical pixels, let's look at 
#the Left and Right components of the overlap.

rcParams['figure.figsize'] = (20,8)

tinyL = imgL[:100,widthL-overlap:]
tinyR = imgR[:100,:overlap]

# display images
fig, ax = plt.subplots(1,2)
ax[0].imshow(tinyL)
ax[1].imshow(tinyR)

In [ ]:
#caluclating the difference

rcParams['figure.figsize'] = (20,10)

#turn both sections to grayscale
tinyL_g = np.dot(tinyL[...,:3], [0.299,0.587,0.114])
tinyR_g = np.dot(tinyR[...,:3], [0.299,0.587,0.114])

#calculate their squared difference
diff = cv2.subtract(tinyL_g,tinyR_g)
diff = cv2.multiply(diff,diff)

#display the squared difference
plt.imshow(diff,cmap='gray')
np.shape(diff)

In [ ]:
#calculating the seam for the small squared difference
#the get_seam() function is defined in EC_CV.py

height, width = diff.shape
my_seam = get_seam(diff)

#show the seam in the squared difference with white pixels
diff_seam = diff.copy()
height, width = diff.shape
for i in range(height):
    diff_seam[i,int(my_seam[i])] = 255 * 255 # This is white squared
plt.imshow(diff_seam,cmap='gray')
print(height)
print(width)

In [ ]:
#using the whole overlap, let's look at its Left and Right components.

rcParams['figure.figsize'] = (20,8)
tinyL = imgL[:,widthL-overlap:]
tinyR = imgR[:,:overlap]

fig, ax = plt.subplots(1,2)
ax[0].imshow(tinyL)
ax[1].imshow(tinyR)

In [ ]:
#calculating their difference.

#turning both sections to grayscale
tinyL_g = np.dot(tinyL[...,:3], [0.299,0.587,0.114])
tinyR_g = np.dot(tinyR[...,:3], [0.299,0.587,0.114])

#calculate their squared difference
diff = cv2.subtract(tinyL_g,tinyR_g)
diff = cv2.multiply(diff,diff)

#display the squared difference
plt.imshow(diff,cmap='gray')
np.shape(diff)

In [ ]:
#let's calculate the seam for the squared difference
rcParams['figure.figsize'] = (20,28)
height, width = diff.shape
my_seam = get_seam(diff)

#show the seam in the squared difference with white pixels
diff_seam = diff.copy()
height, width = diff.shape
for i in range(height):
    diff_seam[i,int(my_seam[i])] = 255 * 255 # This is white squared
plt.imshow(diff_seam,cmap='gray')
print(height)
print(width)

In [ ]:
# First let's create the middle section.
# This is the overlapping section, with pixels from both Left and
# Right components. The seam determines where Left pixels end
# and Right pixels start.

middle = tinyL.copy()    # Start with Left overlap.

# For every row in the middle section, replace Left pixels with 
# Right pixels starting at the seam.
for i in range(height):   
    j = int(my_seam[i])
    while j < width:
        middle[i,j] = tinyR[i,j]
        j += 1
    
# Now let's create a marked middle section to show the seam with red pixels.
middle_marked = middle.copy()
for i in range(height):
    middle_marked[i,int(my_seam[i])] = np.array((255,0,0))
        
rcParams['figure.figsize'] = (20,28)        
plt.imshow(middle_marked,cmap='gray')

In [ ]:
# Now let's create the stitched image consisting of:
#    The Leftmost Image (minus the overlap) 
#       + The Middle Section with the seam in red 
#          + The Rightmost Section (minus the overlap)

cut_stitch = np.concatenate((imgL[:,:widthL-overlap], middle_marked), axis=1)
cut_stitch = np.concatenate((cut_stitch, imgR[:,overlap:]), axis=1)
plt.imshow(cut_stitch)

In [ ]:
#creating the stitched image, not showing the seam.

cut_stitch = np.concatenate((imgL[:,:widthL-overlap], middle), axis=1)
cut_stitch = np.concatenate((cut_stitch, imgR[:,overlap:]), axis=1)
plt.imshow(cut_stitch)

In [ ]:
#displaying both zoomed-in results

rcParams['figure.figsize'] = (20,10)

tiny_stitch = stitch[int(heightL/2)-overlap*2:int(heightL/2)+overlap*2,widthL-overlap*2:widthL+overlap]
tiny_cut_stitch = cut_stitch[int(heightL/2)-overlap*2:int(heightL/2)+overlap*2,widthL-overlap*2:widthL+overlap]

fig, ax = plt.subplots(1,2)
ax[0].imshow(tiny_stitch)
ax[1].imshow(tiny_cut_stitch)

In [ ]:
#finally let's save the stiched image into a file
plt.imsave("stitched.bmp",cut_stitch)